# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [49]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import json  # Import the json module to handle JSON data

# import geoviews as gv  # Import for geospatial plotting
# import hvplot.xarray  # noqa
# import xarray as xr

In [50]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,4.02,48,0,5.66,IS,1728132093
1,1,bethel,41.3712,-73.4140,16.48,84,0,0.89,US,1728132094
2,2,port-aux-francais,-49.3500,70.2167,1.30,93,100,4.63,TF,1728132095
3,3,ghadamis,30.1337,9.5007,32.65,21,97,4.30,LY,1728132097
4,4,hadibu,12.6500,54.0333,28.27,74,52,3.14,YE,1728132098


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:
## Code for the map 

# %%capture --no-display

city_map = city_data_df.hvplot.points('Lng',
                                      'Lat',
                                      geo=True,
                                      tiles=True, 
                                      color='City', 
                                      alpha=0.5,
                                      frame_width = 700,
                                      frame_height = 500,
                                      size = 'Humidity',
                                      cmap='Category10'
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
city_data_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,5.550000e+02
mean,277.000000,18.937609,17.599331,19.619676,68.371171,52.515315,3.459063,1.728132e+09
std,160.358972,31.918409,89.201179,10.047449,23.809091,42.131705,2.374210,1.938685e+02
min,0.000000,-54.800000,-176.559700,-10.050000,1.000000,0.000000,0.000000,1.728132e+09
25%,138.500000,-5.145700,-63.450300,12.185000,55.000000,5.000000,1.720000,1.728132e+09
50%,277.000000,22.239400,23.541400,21.980000,75.000000,54.000000,3.000000,1.728132e+09
75%,415.500000,45.307350,88.781950,27.245000,87.000000,100.000000,4.575000,1.728133e+09
max,554.000000,78.218600,179.383300,42.070000,100.000000,100.000000,11.860000,1.728133e+09


In [79]:
# Narrow down cities that fit criteria and drop any results with null values
cond_temp_min = 20
cond_temp_max = 25
cond_max_hum = 55
cond_cloud = 54

my_city_df = (
    (city_data_df['Max Temp'] > cond_temp_min) & 
    (city_data_df['Max Temp'] < cond_temp_max) & 
    (city_data_df['Humidity'] < cond_max_hum) & 
    (city_data_df['Cloudiness'] < cond_cloud)
    )

my_city_df = city_data_df[my_city_df]

# Drop any rows with null values
my_city_df = my_city_df.dropna(how='any')

# Display sample data

print(f"My prefered Conditions: temperature between ({cond_temp_min},{cond_temp_max}), Humidity {cond_max_hum}, Cloudiness {cond_cloud}")
print(f'size of the new DF {my_city_df.shape}')

my_city_df

My prefered Conditions: temperature between (20,25), Humidity 55, Cloudiness 54
size of the new DF (18, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,tsiombe,-25.3000,45.4833,21.99,50,0,6.32,MG,1728132129
36,36,arraial do cabo,-22.9661,-42.0278,23.92,50,40,4.12,BR,1728132132
59,59,shalqar,47.8333,59.6000,21.09,22,0,1.36,KZ,1728132157
80,80,turpan,42.9333,89.1667,21.98,24,0,0.89,CN,1728132185
83,83,el granada,37.5027,-122.4694,23.68,48,20,0.00,US,1728132189
92,92,karaj,35.8355,51.0103,21.88,31,20,4.12,IR,1728132198
126,126,saint-pierre,-21.3393,55.4781,23.82,53,0,8.23,RE,1728132171
206,206,al mahwit,15.4701,43.5448,22.46,45,12,4.42,YE,1728132327
260,260,altud,43.7219,43.8658,24.25,25,0,4.01,RU,1728132391
305,305,serenje,-13.2325,30.2352,23.80,26,40,4.47,ZM,1728132443


### Step 3: Create a new DataFrame called `hotel_df`.

In [80]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the 
# city, country, coordinates, and humidity
hotel_df = my_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
33,tsiombe,MG,-25.3000,45.4833,50,
36,arraial do cabo,BR,-22.9661,-42.0278,50,
59,shalqar,KZ,47.8333,59.6000,22,
80,turpan,CN,42.9333,89.1667,24,
83,el granada,US,37.5027,-122.4694,48,
92,karaj,IR,35.8355,51.0103,31,
126,saint-pierre,RE,-21.3393,55.4781,53,
206,al mahwit,YE,15.4701,43.5448,45,
260,altud,RU,43.7219,43.8658,25,
305,serenje,ZM,-13.2325,30.2352,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [81]:
## TEST 
print(geoapify_key)

8e0f597430e942f0ac38b55cf85ff32b


In [82]:
# # Set parameters to search for a hotel

latitude = -33.0153
longitude = 27.9116

# Set the parameters for the type of place
# categories = "accommodation.hotel"

# Set the parameters for the type of search
limit = 20
radius = 10000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "categories":"accommodation.hotel",
    "filter": filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
# print(json.dumps(response, indent=4, sort_keys=True))




In [72]:
## To TEST
print(response['features'][0]['properties']['address_line1'])
print(response['features'][0]['properties']['address_line2'])

# print(response["features"][0]["properties"]['address_line1'])
# print(response["features"][0]["properties"]["address_line2"])

Inverleith Terrace
Quigney, East London, 5201, South Africa


In [83]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation.hotel',
    "apiKey" : geoapify_key
    }


# Print a message to follow up the hotel search

print("Starting hotel search")

# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame

    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    # latitude = -33.0153
    # longitude = 27.9116

    # Add the current city's latitude and longitude to the params dictionary

    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL

    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary

    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format

    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]['address_line1']

    except (KeyError, IndexError):

        # If no hotel is found, set the hotel name as "No hotel found".

        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results

    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    # print(f"{hotel_df.loc[index, 'City']} - {latitude}, {longitude}")

# Display sample data

hotel_df

Starting hotel search
tsiombe - nearest hotel: N 10
arraial do cabo - nearest hotel: Rua Rodrigues Alves
shalqar - nearest hotel: Beknur
turpan - nearest hotel: Oriental
el granada - nearest hotel: Beach House
karaj - nearest hotel: هتل مرمر
saint-pierre - nearest hotel: Tropic Hotel
al mahwit - nearest hotel: فندق الماجد
altud - nearest hotel: No hotel found
serenje - nearest hotel: Mogan Guesthouse
cayeli - nearest hotel: No hotel found
aktau - nearest hotel: Гульдана (the real location)
gignac - nearest hotel: Le Vieux Moulin
vail - nearest hotel: Hampton Inn & Suites Tucson Tech Park
san jose de jachal - nearest hotel: San Martin
delijan - nearest hotel: هتل سبز کارون
infantes - nearest hotel: La Gavilla
tagta - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
33,tsiombe,MG,-25.3000,45.4833,50,N 10
36,arraial do cabo,BR,-22.9661,-42.0278,50,Rua Rodrigues Alves
59,shalqar,KZ,47.8333,59.6000,22,Beknur
80,turpan,CN,42.9333,89.1667,24,Oriental
83,el granada,US,37.5027,-122.4694,48,Beach House
92,karaj,IR,35.8355,51.0103,31,هتل مرمر
126,saint-pierre,RE,-21.3393,55.4781,53,Tropic Hotel
206,al mahwit,YE,15.4701,43.5448,45,فندق الماجد
260,altud,RU,43.7219,43.8658,25,No hotel found
305,serenje,ZM,-13.2325,30.2352,26,Mogan Guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [89]:
%%capture --no-display

## Code for the map 

city_map2 = hotel_df.hvplot.points('Lng',
                                      'Lat',
                                      geo=True,
                                      tiles=True, 
                                      color='City', 
                                      alpha=0.5,
                                      frame_width = 700,
                                      frame_height = 500,
                                      size = 'Humidity',
                                      cmap='Category10',
                                      hover_cols=['Country','Hotel Name']
)

# Display the map
city_map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)